# Retrieval

LangChain에서 Retrieval은 외부 데이터에서 관련 정보를 찾아 LLM에 전달하는 역할을 한다. 주요 구성 요소는 다음과 같다.

- **Document Loader**: 다양한 원본 데이터를 LangChain 표준 문서 객체로 변환한다.
- **Text Splitter**: 긴 문서를 작은 청크로 분할해 검색 효율을 높인다.
- **Embedding Model**: 텍스트를 의미 기반 벡터로 변환한다.
- **Vector Store**: 임베딩된 벡터를 저장하고 유사도 기반 검색을 지원한다.
- **Retriever**: 쿼리에 대해 관련 문서를 찾아주는 표준 인터페이스를 제공한다.
- **Retrieval Chain**: 검색된 문서를 LLM에 전달해 답변을 생성하는 체인 구조를 제공한다.

이렇게 각 모듈이 결합되어, 외부 데이터 기반의 효과적인 검색 및 답변 생성이 가능하다.

### 환각 Hallucination
LLM이 실제 근거 없이 그럴듯해 보이는 정보를 생성하는 현상이다.

**주요 원인**

1. **학습 데이터 한계**

   * 모델이 학습한 데이터에 해당 정보가 없거나 부족할 때 발생한다.
2. **확률적 생성 과정**

   * 토큰 예측 시 언어적 일관성을 우선하다 보니, 사실 여부가 검증되지 않은 내용을 생성한다.
3. **프롬프트 모호성**

   * 지시가 불명확하거나 맥락이 부족하면 모델이 관련 없는 정보를 보충·왜곡한다.

**대표 사례**

* 존재하지 않는 논문·저자명을 인용함.
* 역사적·과학적 사실을 잘못 기술함.
* 실행 불가능하거나 비효율적인 코드 제안.


**완화 방안**

1. **지식 기반 검색 결합**

   * Retrieval-Augmented Generation(RAG) 방식으로 외부 문서·데이터베이스에서 실시간 근거를 가져와 보강한다.
2. **프롬프트 구체화**

   * “출처를 함께 제시해 달라” 등 명시적 요청을 통해 근거 표기를 유도한다.
3. **후처리 검증**

   * 생성 결과를 룰 기반 검증 또는 전문가 리뷰를 통해 교차 확인한다.
4. **모델 파인튜닝 및 앙상블**

   * 도메인 특화 데이터로 추가 학습하거나, 룰 기반 시스템과 결합하여 정확도를 높인다.


## Document

Document는 LangChain 프레임워크에서 다양한 데이터 소스(예: 텍스트 파일, PDF, 웹페이지 등)로부터 불러온 정보를 표준화된 객체로 표현하는 핵심 데이터 구조이다. 이 객체는 언어 모델(LLM)이 외부 데이터를 이해하고 처리할 수 있도록 도와준다.

**Document 객체의 구조**
1. page_content: 문서의 실제 내용을 담고 있는 문자열(str)이다. 예를 들어, 텍스트 파일의 본문이나 PDF의 텍스트 등이 여기에 저장된다.
2. metadata: 문서에 대한 부가 정보를 담는 딕셔너리(dict) 형태의 속성이다. 예를 들어, 파일 경로, 페이지 번호, 작성자, 데이터 출처 등 다양한 메타데이터를 저장할 수 있다.


**Document의 역할과 활용**
1. 표준화된 데이터 구조: 다양한 포맷의 데이터를 일관된 방식으로 표현하여, LLM이 손쉽게 접근하고 활용할 수 있도록 한다.
2. 문서 처리의 기본 단위: LangChain의 문서 로더(Document Loader)는 파일, 웹, 데이터베이스 등 여러 소스에서 데이터를 읽어와 Document 객체로 변환한다.
3. 청크 단위 분할: 대용량 문서는 작은 단위(청크)로 쪼개어 각각의 Document로 저장하고, 검색 및 임베딩 처리에 활용한다.



In [1]:
!pip install langchain langchain-community langchain-openai langchain-huggingface pypdf tiktoken faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9

In [2]:
from langchain_core.documents import Document

doc = Document(
    page_content='이것은 문서의 내용입니다.',
    metadata={
        'soucrce': 'ABC pdf',
        'page': 33,
        'author': '김철수',
        'date': '2022-02-02'
    }
)
doc

Document(metadata={'soucrce': 'ABC pdf', 'page': 33, 'author': '김철수', 'date': '2022-02-02'}, page_content='이것은 문서의 내용입니다.')

## Document Loader
https://python.langchain.com/api_reference/core/document_loaders.html

https://python.langchain.com/api_reference/community/document_loaders.html

Document Loader는 다양한 데이터 소스에서 데이터를 읽어와 Document 객체로 변환하는 역할을 한다. 예를 들어, PDFLoader, CSVLoader, TextLoader 등 다양한 종류가 존재하며, 각기 다른 파일 형식을 Document 객체로 표준화한다.

Document Loader는 데이터 소스별로 특화된 클래스를 제공하며, 문서를 로드한 후 LangChain에서 사용하는 표준 형식으로 변환해준다.

1. **다양한 데이터 소스 지원**  
   Document Loader는 파일 시스템, 클라우드 스토리지, 데이터베이스, 웹 등 다양한 데이터 소스에서 데이터를 로드할 수 있도록 설계되었다.
   
2. **표준화된 출력 형식**  
   로드된 문서는 LangChain에서 사용하는 `Document` 객체로 변환된다. `Document` 객체는 다음과 같은 필드를 포함한다:
   - `page_content`: 문서 본문 내용
   - `metadata`: 문서와 관련된 메타데이터 (예: 파일 이름, URL, 작성자 등)

3. **플러그인 기반 확장 가능**  
   사용자 정의 데이터 소스 로더를 쉽게 구현하고 LangChain에 통합할 수 있다.

**주요 Document Loader 예시**

| Loader 이름        | 설명                                                              |
|--------------------|-------------------------------------------------------------------|
| `PyPDFLoader`      | PDF 문서를 로드하며 텍스트를 추출해 Document 형식으로 변환한다.     |
| `TextLoader`       | 일반 텍스트 파일을 로드한다.                                      |
| `UnstructuredFileLoader` | 비구조적 데이터를 로드하여 구조화된 텍스트로 변환한다.           |
| `CSVLoader`        | CSV 파일에서 데이터를 로드하며 행(row)을 Document로 처리한다.      |
| `WebBaseLoader`    | 웹 페이지 데이터를 크롤링하여 Document로 로드한다.                |


### WebBaseLoader


In [3]:
from langchain_community.document_loaders import WebBaseLoader

url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url, header_template={"user_agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"})
loader

In [4]:
docs: list[Document] = loader.load()
print(len(docs))
print(docs[0].metadata)
print(docs[0].page_content[2000:2500])

1
{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}
문서는 한국어 위키백과의 정책입니다.이것은 모든 사용자들이 일반적으로 따라야 하는 널리 인정된 기준입니다. 문서의 변경은 총의를 반영해야 합니다.단축백:정책백:지침백:규칙
요약: 위키백과의 정책과 지침은 위키백과 공동체가 지켜야 할 규범과 사례를 모아둔 것을 의미합니다. 본 정책은 정책과 지침이 어떻게 만들어지고 유지하며 지켜야 하는 지에 대해 설명합니다.
정책과 지침(목록)
원칙
다섯 원칙
규칙에 얽매이지 마세요

콘텐츠 정책(핵심)
중립적 시각
확인 가능
독자 연구 금지
위키백과에 대한 오해

행동 정책
총의
분쟁 해결
편집 분쟁
삭제 정책
차단 정책
법적 위협 금지
인신 공격 금지
문서의 소유권
계정 이름
문서 훼손

기타 정책 분류
집행 정책
법적 정책
절차 정책
생존 인물의 전기

목록
정책 목록
지침 목록
vte
위키백과의 정책(Policy)과 지침(Guideline)은 위키미디어 공동체가 모범적인 활동 방식, 분쟁 해결, 문서의 표준 작성 기준 등을 제시하기 위해 개발한
